# Getting Setup
This section goes through various steps to get setup for outlier detection. For more details see the ingesting ISIS control networks tutorial.

## Imports

In [1]:
# Required for autocnet imports
import os
os.environ['ISISROOT'] = '/usgs/cpkgs/anaconda3_linux/envs/isis4.2.0'

# Autocnet Imports
from autocnet.graph.network import NetworkCandidateGraph
from autocnet.graph.edge import NetworkEdge
from autocnet.io.db.model import Matches, Points
from autocnet.transformation.roi import Roi

# Helpful Python Modules
import matplotlib.pyplot as plt  #     plotting package
import numpy as np               #     numerical computing package

## Setting up the NetworkCandidateGraph

### Config

The config various settings that autocnet will use when connecting to other services. Primarily, the config is used to define:

- The database your NetworkCandidateGraph will use
- The redis queue and slurm settings for cluster based processing
- The spatial reference system for geometries such as image footprints

In [2]:
config_path = 'config.yml'

### Loading the control network
This cell will check if the database your config file points to already has a control network ingested in it. If it doesn't then it goes through the steps from the ingesting ISIS control networks tutorial.

In [3]:
data_directory = "/work/projects/control_network_metrics/tutorials/isis_ingestion"
lis_path = os.path.join(data_directory, "apollo_lronac_cubes.lis")
cnet_path = os.path.join(data_directory, "AS15_landingsite_apollolro_jig1.net")

ncg = NetworkCandidateGraph()
ncg.config_from_file(config_path)
ncg.from_database()
if len(ncg) == 0:
    print(f'Ingesting control network {cnet_path}.')
    ncg = NetworkCandidateGraph.from_cnet(cnet_path, lis_path, config_path)
else:
    print('Network already in database')

OperationalError: (psycopg2.OperationalError) ERROR:  client_login_timeout (server down)

(Background on this error at: http://sqlalche.me/e/13/e3q8)

### Check network
Look at the graph of the network to ensure it isn't malformed. Each image in the network is represented by a node and overlapping images have an edge between their nodes. We will be doing pair-wise outlier detection, so we will check each edge in the network graph

In [ ]:
ncg.plot()

### Getting the pairwise image matches
The first process we need to do is collect all of the pairwise matches between images. Our control network currently contains control points and all of the measurements of them. We need to convert these multi-image relationships into all of the common points between each pair of images.

To do this, we're going to use Autocnet's apply function to run parallel processing on each edge of the NetworkCandidateGraph

In [ ]:
?NetworkCandidateGraph.apply

#### SLURM parameters
These parameters will be used when creating SLURM jobs for cluster processing via the apply function. Depending on the complexity of the jobs you are running, you may want to change the walltime and arraychunk parameters.

In [ ]:
walltime="00:30:00"
log_dir = '/scratch/jmapel/autocnet_tut/logs'
arraychunk=75
chunksize=16723

#### Convert control measures and points to image matches
This cell uses apply to run the network_to_matches function on each edge. It is very important that this function only get run once per edge or it will add duplicate matches. So, this cell also contains a check that skips the function if the database already has matches in it.

In [ ]:
?NetworkEdge.network_to_matches

In [ ]:
with ncg.session_scope() as session:
    num_matches = session.query(Matches).count()
if num_matches == 0:
    print("Loading matches table")
    njobs = ncg.apply('network_to_matches', 
                      on='edges',
                      # SLURM kwargs
                      walltime=walltime,
                      log_dir=os.path.join(log_dir, 'matches'),
                      arraychunk=arraychunk,
                      chunksize=chunksize)
else:
    print("Matches table already populated")

### Looking at the matches
The pairwise image matches are stored on the edges of the graph and can be accessed via the networkX graph or the matches table in the database

In [ ]:
for source, dest, edge in ncg.edges(data='data'):
    print(f'Edge ({source}, {dest}) has {len(edge.matches)} matches')

## Outlier Detection

### Reprojective Error
These two functions check for outliers by attempting to reproject measures between images. Any pair of measures that do not repoject to each other wth in a given tolerance are flagged.

In [ ]:
?NetworkEdge.compute_fundamental_matrix

In [ ]:
?NetworkEdge.compute_homography

These cells also demonstrate how to pass kwargs through apply. Any kwargs that are not specific to the apply function are passed on to the function being applied.

In [ ]:
njobs = ncg.apply('compute_fundamental_matrix', 
                  on='edges',
                  # homography kwargs
                  method='mle',
                  reproj_threshold=5,
                  # SLURM kwargs
                  walltime=walltime,
                  log_dir=os.path.join(log_dir, 'fundamental'),
                  arraychunk=arraychunk,
                  chunksize=chunksize)

In [ ]:
njobs = ncg.apply('compute_homography', 
                  on='edges',
                  # fundamental matrix kwargs
                  method='lmeds',
                  reproj_threshold=5,
                  # SLURM kwargs
                  walltime=walltime,
                  log_dir=os.path.join(log_dir, 'homography'),
                  arraychunk=arraychunk,
                  chunksize=chunksize)

You can check the queue_length property on the NetworkCandidateGraph object to see how many jobs are either waiting to be processed or in process. You can also use the squeue command on a command line or in your notebook to check what jobs slurm has.

In [ ]:
ncg.queue_length

In [ ]:
!squeue -u jmapel

### Looking at the results
The reprojective error checks add a property to each called masks. This is a Pandas dataframe that contains a column for each check that has been done on the edge. If the row for a match has a true in it, then that match passed the column's check. Conversely, if the row for a match has a false in it, then that match failsed the column's check. We can use some dataframe techniques to look at our results

In [ ]:
for source, dest, edge in ncg.edges(data="data"):
    num_matches = len(edge.matches)
    # MLE requires at least 8 points so skip anything with too few
    if num_matches < 8:
        continue
    print(f'edge ({source}, {dest})')
    print('num matches:', num_matches)
    print('passed homography:', sum(edge.masks['homography']))
    print('passed fundamental:', sum(edge.masks['fundamental']))
    print('too few matches to compute fundamental matrix')
    print('')

## Digging in on edge (9, 7)
Edge (9, 7) has a lot of matches but many failures; let's take a closer look at it.

In [ ]:
edge_9_7 = ncg.edges[(9, 7)]['data']
image_9 = ncg.nodes[9]
image_7 = ncg.nodes[7]
print(edge_9_7)
print(image_9)
print(image_7)

### Looking at the matches that failed outlier detection
We can use the masks dataframe to index the matches dataframe on our edge and see the matches that failed each check

In [ ]:
failed_fundamental = edge_9_7.matches.loc[(~edge_9_7.masks[['fundamental']]).all(axis=1)]
failed_fundamental

In [ ]:
failed_homography = edge_9_7.matches.loc[(~edge_9_7.masks[['homography']]).all(axis=1)]
failed_homography

We can even use some techniques to see the matches that failed both checks

In [ ]:
failed_both = edge_9_7.matches.loc[(~edge_9_7.masks[['fundamental', 'homography']]).all(axis=1)]
failed_both

### Viewing individual matches
Let's take a closer look at the matches that failed both checks. We're going to use the Roi (Region of Interest) objects in Autocnet that allow you to look at a small portion of an image.

In [ ]:
roi_size = 25
for idx, match in failed_both.iterrows():
    with ncg.session_scope() as session:
        point_name = session.query(Points).filter(Points.id == match["point_id"]).first().identifier

    source_image = ncg.nodes[match['source']]['data']
    dest_image = ncg.nodes[match['destination']]['data']
    source_roi = Roi(source_image.geodata, match['source_x'], match['source_y'], size_x=roi_size, size_y=roi_size)
    dest_roi = Roi(dest_image.geodata, match['destination_x'], match['destination_y'], size_x=roi_size, size_y=roi_size)

    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.suptitle(point_name)
    ax1.imshow(source_roi.array, cmap='gray')
    ax1.plot(source_roi.center[0] + source_roi.axr, source_roi.center[1] + source_roi.ayr, 'ro')
    ax1.title.set_text(os.path.split(source_image['image_name'])[-1])
    ax2.imshow(dest_roi.array, cmap='gray')
    ax2.plot(dest_roi.center[0] + dest_roi.axr, dest_roi.center[1] + dest_roi.ayr, 'bo')
    ax2.title.set_text(os.path.split(dest_image['image_name'])[-1])
    fig.show()